In [1]:
import pandas as pd

In [12]:
matches = pd.read_csv('combined_seasons.csv')

In [13]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,sh,sot,dist,pk,pkatt,season,team,xg,xga,fk
0,2014-08-16,15:00,Premier League,Matchweek 1,Sat,Away,D,2.0,2.0,West Brom,...,7.0,2.0,NaN,0,0,2016,Sunderland,NaN,NaN,NaN
1,2014-08-16,17:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Crystal Palace,...,14.0,6.0,NaN,0,0,2016,Arsenal,NaN,NaN,NaN
2,2014-08-16,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,1.0,Tottenham,...,16.0,4.0,NaN,0,1,2016,West Ham United,NaN,NaN,NaN
3,2014-08-16,17:30,Premier League,Matchweek 1,Sat,Away,L,1.0,2.0,Arsenal,...,4.0,2.0,NaN,0,0,2016,Crystal Palace,NaN,NaN,NaN
4,2014-08-16,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,1.0,Hull City,...,19.0,6.0,NaN,0,1,2016,Queens Park Rangers,NaN,NaN,NaN


In [14]:
matches.shape

(6916, 28)

In [19]:
matches["team"].value_counts()
# Some teams have gotten promoted or relegated over the years but we are only intereseted in the teams
# in current 2024-2025 season


,count
team,
Liverpool,346
West Ham United,346
Crystal Palace,346
Manchester United,346
Everton,346
Chelsea,346
Manchester City,346
Tottenham Hotspur,345
Arsenal,345


In [20]:
matches["round"].value_counts()

,count
round,
Matchweek 1,200
Matchweek 3,200
Matchweek 2,200
Matchweek 4,196
Matchweek 29,180
Matchweek 23,180
Matchweek 24,180
Matchweek 25,180
Matchweek 26,180


In [21]:
# Change date type to time
matches["date"] = pd.to_datetime(matches["date"])

In [22]:
matches["venue_cat"]= matches["venue"].astype("category").cat.codes

In [23]:
matches['opp_cat'] = matches['opponent'].astype('category').cat.codes

In [24]:
matches["hour"]= matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [33]:
matches["day_cat"]= matches["date"].dt.dayofweek

In [34]:
#Target is a WIN
matches["target"] = (matches["result"] == "W").astype("int")

In [249]:
from sklearn.ensemble import HistGradientBoostingClassifier


In [250]:
# Initializing random forest tree model
hg= HistGradientBoostingClassifier()

In [251]:
#Training with all matches before the year 2024
train = matches[matches["date"] < '2024-01-01']

In [252]:
test = matches[matches["date"] > '2024-01-01']

In [253]:
predictors = ["venue_cat", "opp_cat", "hour", "day_cat","sh","pk","sot","xg","xga"]

In [254]:
hg.fit(train[predictors], train["target"])

HistGradientBoostingClassifier()

In [255]:
preds = hg.predict(test[predictors])

In [256]:
from sklearn.metrics import accuracy_score

In [257]:
acc = accuracy_score(test["target"], preds)

In [258]:
acc

0.7647058823529411

In [259]:
#create a dataframe
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [260]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])
#

predicted,0,1
actual,,
0,234,43
1,61,104


In [261]:
from sklearn.metrics import precision_score

In [262]:
precision_score(test["target"], preds)


0.7074829931972789

In [263]:
#Create one dataframe for each team
grouped_matches = matches.groupby("team")

In [264]:
#Checking their form in the previous matches
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    #closed left to now include current week
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [318]:
cols = ["gf", "sh", "sot", "pk",]
new_cols = [f"{c}_rolling" for c in cols]


In [319]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [320]:
matches_rolling

date   time            comp         round  \
team                                                                           
Arsenal                 71   2014-09-13  12:45  Premier League   Matchweek 4   
                        86   2014-09-20  15:00  Premier League   Matchweek 5   
                        103  2014-09-27  17:30  Premier League   Matchweek 6   
                        134  2014-10-05  14:05  Premier League   Matchweek 7   
                        140  2014-10-18  15:00  Premier League   Matchweek 8   
...                                 ...    ...             ...           ...   
Wolverhampton Wanderers 6796 2024-05-11  15:00  Premier League  Matchweek 37   
                        6837 2024-05-19  16:00  Premier League  Matchweek 38   
                        6842 2024-08-17  15:00  Premier League   Matchweek 1   
                        6875 2024-08-25  14:00  Premier League   Matchweek 2   
                        6889 2024-08-31  15:00  Premier League   Matchweek 3   

                              day venue result   gf   ga         opponent  \
team                                                                        
Arsenal                 71    Sat  Home      D  2.0  2.0  Manchester City   
                        86    Sat  Away      W  3.0  0.0      Aston Villa   
                        103   Sat  Home      D  1.0  1.0        Tottenham   
                        134   Sun  Away      L  0.0  2.0          Chelsea   
                        140   Sat  Home      D  2.0  2.0        Hull City   
...                           ...   ...    ...  ...  ...              ...   
Wolverhampton Wanderers 6796  Sat  Home      L  1.0  3.0   Crystal Palace   
                        6837  Sun  Away      L  0.0  2.0        Liverpool   
                        6842  Sat  Away      L  0.0  2.0          Arsenal   
                        6875  Sun  Home      L  2.0  6.0          Chelsea   
                        6889  Sat  Away      D  1.0  1.0  Nott'ham Forest   

                              ...  venue_cat  opp_cat hour day_code target  \
team                          ...                                            
Arsenal                 71    ...          1       18   12        5      0   
                        86    ...          0        1   15        5      1   
                        103   ...          1       29   17        5      0   
                        134   ...          0        7   14        6      0   
                        140   ...          1       12   15        5      0   
...                           ...        ...      ...  ...      ...    ...   
Wolverhampton Wanderers 6796  ...          1        8   15        5      0   
                        6837  ...          0       16   16        6      0   
                        6842  ...          0        0   15        5      0   
                        6875  ...          1        7   14        6      0   
                        6889  ...          0       22   15        5      0   

                             day_cat gf_rolling  sh_rolling  sot_rolling  \
team                                                                       
Arsenal                 71         5   1.666667   17.000000     5.000000   
                        86         5   1.666667   17.333333     5.000000   
                        103        5   2.000000   16.000000     5.000000   
                        134        6   2.000000   13.333333     5.000000   
                        140        5   1.333333   11.666667     3.000000   
...                              ...        ...         ...          ...   
Wolverhampton Wanderers 6796       5   1.000000   10.000000     3.333333   
                        6837       6   1.333333    9.666667     4.666667   
                        6842       5   0.666667    6.666667     3.666667   
                        6875       6   0.333333    9.000000     4.333333   
                        6889       5   0.666667    8.333333     3

In [321]:
matches_rolling = matches_rolling.droplevel('team')

In [322]:
#add new indices for the rows
matches_rolling.index = range(matches_rolling.shape[0])

In [323]:
def make_predictions(data,predictors):
  train = data[data["date"] < '2024-01-01']
  test = data[data["date"] > '2024-01-01']
  hg.fit(train[predictors], train["target"])
  preds = hg.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
  precision = precision_score(test["target"], preds)
  accuracy = accuracy_score(test["target"], preds)
  return combined, precision, accuracy

In [324]:
combined, precision , accuracy = make_predictions(matches_rolling, predictors + new_cols)

In [325]:
precision

0.7341772151898734

In [326]:
accuracy

0.7927107061503417

In [327]:
combined

,actual,predicted
321,1,1
322,1,0
323,1,1
324,1,1
325,1,1
...,...,...
6809,0,0
6810,0,0
6811,0,0
6812,0,0


In [328]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [329]:
combined

,actual,predicted,date,team,opponent,result
321,1,1,2024-01-20,Arsenal,Crystal Palace,W
322,1,0,2024-01-30,Arsenal,Nott'ham Forest,W
323,1,1,2024-02-04,Arsenal,Liverpool,W
324,1,1,2024-02-11,Arsenal,West Ham,W
325,1,1,2024-02-17,Arsenal,Burnley,W
...,...,...,...,...,...,...
6809,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L
6810,0,0,2024-05-19,Wolverhampton Wanderers,Liverpool,L
6811,0,0,2024-08-17,Wolverhampton Wanderers,Arsenal,L
6812,0,0,2024-08-25,Wolverhampton Wanderers,Chelsea,L


In [330]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Sheffield United":"Sheffield Utd","Queens Park Rangers":"QPR","Newcastle United": "Newcastle Utd","West Bromwich Albion": "West Brom","Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"}
mapping = MissingDict(**map_values)

In [331]:
mapping["West Ham United"]

'West Ham'

In [332]:
combined["new_team"] = combined["team"].map(mapping)

In [333]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [334]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,0,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
2,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal,0,0,Liverpool,Arsenal,L,Liverpool
3,1,1,2024-02-11,Arsenal,West Ham,W,Arsenal,0,0,West Ham United,Arsenal,L,West Ham
4,1,1,2024-02-17,Arsenal,Burnley,W,Arsenal,0,0,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L,Wolves,1,0,Crystal Palace,Wolves,W,Crystal Palace
410,0,0,2024-05-19,Wolverhampton Wanderers,Liverpool,L,Wolves,1,1,Liverpool,Wolves,W,Liverpool
411,0,0,2024-08-17,Wolverhampton Wanderers,Arsenal,L,Wolves,1,1,Arsenal,Wolves,W,Arsenal
412,0,0,2024-08-25,Wolverhampton Wanderers,Chelsea,L,Wolves,1,1,Chelsea,Wolves,W,Chelsea


In [338]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

,count
actual_x,
1,107
0,29


In [ ]:
# from looking at the matches in which the algorithm predicted on both sides
# and got .787 precision